# ArcFace: Additive Angular Margin Loss for Deep Face Recognition.  
[click here](https://arxiv.org/abs/1801.07698)   
# Not Ready Yet 

In [47]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import math
from tensorflow.keras import backend as K
import os
from skimage.transform import resize
from skimage.io import imread

In [48]:
class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, X, labels, batch_size) :
    self.X = X
    self.labels = labels
    self.batch_size = batch_size
    self.shuffle = True
    self.on_epoch_end()

  def __iter__(self):
    """Create a generator that iterate over the Sequence."""
    for item in (self[i] for i in range(len(self))):
      yield item  
    
  def __len__(self) :
    #self.on_epoch_end()
    result = (np.ceil(len(self.X) / float(self.batch_size))).astype(np.int)
    return result

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.labels))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)
  
  def __getitem__(self, idx) :
    indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
    batch_x = [self.X[k] for k in indexes]
    batch_y = [self.labels[k] for k in indexes] 
    X = np.array(batch_x, dtype=np.int32)
    Y = np.array(batch_y)
    #print(Y.shape)
    X = np.concatenate((X, Y), axis=1)
     
    return X, Y


class My_Custom_GeneratorURLs(tf.keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size, height, width, num_channels = 3) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    self.shuffle = True
    self.on_epoch_end()
    self.height = height
    self.width = width
    self.num_channels = num_channels

  def __iter__(self):
    """Create a generator that iterate over the Sequence."""
    for item in (self[i] for i in range(len(self))):
      yield item  
    
  def __len__(self) :
    #self.on_epoch_end()
    result = (np.floor(len(self.image_filenames) / float(self.batch_size))).astype(np.int)  # do not  use ceil  because the histogram don know
    return result

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.labels))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)
  
  def __getitem__(self, idx) :
    #global height_reshaped, width_reshaped
    indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
    b_size = len(indexes)

    batch_x = [self.image_filenames[k] for k in indexes] 
    batch_y = [self.labels[k] for k in indexes]
    #  255.0 * resize(imread(str(file_name)), (height_reshaped, width_reshaped, 3))
    X = np.array([ 255.0 * resize(imread(str(file_name)), (self.height, self.width, 3))     #
               for file_name in batch_x], dtype=np.int32)
    
    Y = np.array(batch_y)
    X = np.reshape(X, (b_size, self.num_channels * self.height * self.width))

    #Y = np.reshape(Y, (-1,1))
    
    X = np.concatenate((X, Y), axis=1)
    return X, Y 


In [49]:
class ArcFace(layers.Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = tf.keras.regularizers.get(regularizer)

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape)
        self.W = self.add_weight(name='W',
                                shape=(input_shape[-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, x, y):
        #x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


In [50]:
num_patterns = 2000
num_features = 2
mu, sigma = 0, 0.5
num_classes = 2
num_hidden_neurons = 4
Xred = np.random.normal(mu, sigma, (num_patterns, num_features)) + np.array([1,1])
Yred = np.zeros(num_patterns, dtype=int)
Xblue = np.random.normal(mu, sigma, (num_patterns, num_features)) + np.array([-1,-1])
Yblue = np.ones(num_patterns, dtype=int)
X = np.concatenate((Xred, Xblue), axis=0)
Y = np.concatenate((Yred, Yblue), axis=0)
Y = np.reshape(Y, (-1,1))

Y_hot = np.squeeze(np.eye(num_classes)[Y.reshape(-1)])
print(X.shape)
print(Y.shape)
print(Y_hot.shape)
print(Y)

(4000, 2)
(4000, 1)
(4000, 2)
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [51]:
inputs = tf.keras.Input(shape=(num_features+num_classes))
real_input = inputs[:,0:num_features]
print(real_input.shape)
real_output = inputs[:,num_features: num_features+num_classes]
print(real_output.shape)
output_1 = layers.Dense(4, activation='relu')(real_input)
output_2 = layers.Dense(4, activation='relu')(output_1)
predictions = ArcFace(num_classes)(output_2, real_output)  
model = tf.keras.Model(inputs, predictions)

(None, 2)
(None, 2)


In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_hot, test_size=0.1, random_state=34)
batch_size = 100
num_epochs = 50
my_training_batch_generator = My_Custom_Generator(X_train, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_test, y_test, batch_size)

model.compile(
  'adam',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy']
)

model.fit(
  my_training_batch_generator,
  epochs=num_epochs,
  validation_data = my_validation_batch_generator,
  validation_steps=len(my_validation_batch_generator)  
)

Epoch 1/50
36/36 [==============================] - 1s 22ms/step - loss: 9.4892 - accuracy: 0.2533 - val_loss: 8.4448 - val_accuracy: 0.2500
Epoch 2/50
36/36 [==============================] - 0s 8ms/step - loss: 7.2776 - accuracy: 0.3236 - val_loss: 6.0708 - val_accuracy: 0.4075
Epoch 3/50
36/36 [==============================] - 0s 8ms/step - loss: 6.2371 - accuracy: 0.4881 - val_loss: 5.7843 - val_accuracy: 0.5150
Epoch 4/50
36/36 [==============================] - 0s 7ms/step - loss: 6.0654 - accuracy: 0.4975 - val_loss: 5.6306 - val_accuracy: 0.5150
Epoch 5/50
36/36 [==============================] - 0s 8ms/step - loss: 5.9281 - accuracy: 0.4975 - val_loss: 5.4942 - val_accuracy: 0.5150
Epoch 6/50
36/36 [==============================] - 0s 7ms/step - loss: 5.8179 - accuracy: 0.4978 - val_loss: 5.3917 - val_accuracy: 0.5175
Epoch 7/50
36/36 [==============================] - 0s 8ms/step - loss: 5.7468 - accuracy: 0.5697 - val_loss: 5.3399 - val_accuracy: 0.6500
Epoch 8/50
36/36 [=

In [53]:
classes_texts = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes_texts)
height = 32
width = 32
num_channels = 3
batch_size = 8
X_train = list(((os.path.abspath('./../../assets/cifar10small/'+ name+str(i+1)+'.png')) for name in classes_texts for i in range(7)))
X_test  = list(((os.path.abspath('./../../assets/cifar10small/'+ name+str(7+i+1)+'.png')) for name in classes_texts for i in range(3))) 
y_train = list((j for j in range(10) for i in range(7)))
y_test = list((j for j in range(10) for i in range(3)))

y_train_hot = np.squeeze(np.eye(num_classes)[np.array(y_train).reshape(-1)])
y_test_hot = np.squeeze(np.eye(num_classes)[np.array(y_test).reshape(-1)])

my_training_batch_generator_cifar = My_Custom_GeneratorURLs(X_train, y_train_hot, batch_size, height, width)
my_validation_batch_generator_cifar = My_Custom_GeneratorURLs(X_test, y_test_hot, batch_size, height, width)

#print(y_train)
#print(y_train_hot)


In [54]:
num_features = height * width*  num_channels
inputs = tf.keras.Input(shape=(num_features + num_classes))

#flatten_images = flatten_images.astype('float32')
flatten_images = (inputs[:, 0:num_features] / 255.0) 
# you can reshape output_1 and convert it in the original dataset
#flatten_images = tf.reshape(flatten_images, [-1, height, width, num_channels])
output_1 = layers.Dense(14, activation='relu')(flatten_images)
output_2 = layers.Dense(14, activation='relu')(output_1)

# you can compare how faster is the ArcFace decreasing the loss
#predictions = layers.Dense(num_classes, activation='softmax')(output_2)
predictions = ArcFace(num_classes)(output_2, inputs[:, num_features: num_features + num_classes])  
model = tf.keras.Model(inputs, predictions)

In [55]:
num_epochs = 500
model.compile(
  'adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy']
)

model.fit(
  my_training_batch_generator_cifar,
  epochs=num_epochs,
  validation_data = my_validation_batch_generator_cifar  
)

17
Epoch 362/500
8/8 [==============================] - 1s 85ms/step - loss: 0.0551 - accuracy: 1.0000 - val_loss: 31.3938 - val_accuracy: 0.0417
Epoch 363/500
8/8 [==============================] - 1s 83ms/step - loss: 0.0543 - accuracy: 1.0000 - val_loss: 31.9880 - val_accuracy: 0.0000e+00
Epoch 364/500
8/8 [==============================] - 1s 89ms/step - loss: 0.0553 - accuracy: 1.0000 - val_loss: 31.5472 - val_accuracy: 0.0000e+00
Epoch 365/500
8/8 [==============================] - 1s 80ms/step - loss: 0.0560 - accuracy: 1.0000 - val_loss: 30.8393 - val_accuracy: 0.0417
Epoch 366/500
8/8 [==============================] - 1s 85ms/step - loss: 0.0544 - accuracy: 1.0000 - val_loss: 33.0053 - val_accuracy: 0.0417
Epoch 367/500
8/8 [==============================] - 1s 87ms/step - loss: 0.0538 - accuracy: 1.0000 - val_loss: 33.9903 - val_accuracy: 0.0000e+00
Epoch 368/500
8/8 [==============================] - 1s 81ms/step - loss: 0.0521 - accuracy: 1.0000 - val_loss: 31.7065 - val_a

In [56]:

height = 32
width = 32
num_features = height * width*  num_channels
my_training_batch_generator_cifar = My_Custom_GeneratorURLs(X_train, y_train, batch_size, height, width)
my_validation_batch_generator_cifar = My_Custom_GeneratorURLs(X_test, y_test, batch_size, height, width)

inputs = tf.keras.Input(shape=(num_features + num_classes))
flatten_images = inputs[:, 0:num_features]/ 255.0
# you can reshape output_1 and convert it in the original dataset
flatten_images = tf.reshape(flatten_images, [-1, height, width, num_channels])

flatten_images = tf.keras.applications.ResNet152V2(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=( height, width, num_channels),
    pooling=None,
    classes=num_classes,
    classifier_activation=None,
)(flatten_images)


# you can compare how faster is the ArcFace decreasing the loss
#predictions = layers.Dense(num_classes, activation='softmax')(output_2)
predictions = ArcFace(num_classes)(flatten_images, inputs[:, num_features: num_features + num_classes])  
model = tf.keras.Model(inputs, predictions)

In [57]:
num_epochs = 50
model.compile(
  'adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy']
)

model.fit(
  my_training_batch_generator_cifar,
  epochs=num_epochs,
  validation_data = my_validation_batch_generator_cifar,
  validation_steps=len(my_validation_batch_generator)  
)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)